# Conditional GAN on CIFAR-10 — Training Notebook

This notebook trains the improved cGAN model on Google Colab with a T4 GPU.

**Steps:**
1. Clone the repo and install dependencies
2. Verify GPU is available
3. Train the model (200 epochs, ~2-3 hours on T4)
4. Evaluate results
5. Download checkpoints and results

## Step 1: Setup — Clone repo and install dependencies

Run this cell first. It clones the project and installs the required packages.

In [ ]:
# Clone the repository
!git clone https://github.com/hussnain-raza-dr/Scientific-Computing-Project-Master.git
%cd Scientific-Computing-Project-Master

# Checkout the improved branch
!git checkout claude/improve-part-a-results-DIMqN

# Install dependencies (PyTorch is already installed on Colab with CUDA)
!pip install -q pyyaml matplotlib torchvision

## Step 2: Verify GPU is available

You should see `cuda` and `Tesla T4` (or similar). If you see `cpu`, go to
**Runtime > Change runtime type** and select **T4 GPU**.

In [ ]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("WARNING: No GPU detected! Go to Runtime > Change runtime type > T4 GPU")

## Step 3: Train the improved model

This trains for 200 epochs with all the improvements:
- TTUR (separate G/D learning rates)
- EMA (Exponential Moving Average) of generator weights
- Cosine LR scheduling
- Instance noise injection (decays over training)
- Orthogonal weight initialization

On a T4 GPU, each epoch takes roughly 40-60 seconds.

Checkpoints are saved every 25 epochs, so if Colab disconnects you can resume.

In [ ]:
# Train from scratch
!python training/train.py --config config/improved_config.yaml --arch improved

### Resume from checkpoint (if Colab disconnected)

If your Colab session disconnected, you can resume training from the last checkpoint.
Uncomment the cell below and update the epoch number to your last saved checkpoint.

In [ ]:
# # Resume from checkpoint — uncomment and update epoch number if needed
# !python training/train.py \
#     --config config/improved_config.yaml \
#     --arch improved \
#     --resume checkpoints/improved_epoch_0100.pt

## Step 4: View training results

Let's look at the training curves and generated samples.

In [ ]:
from IPython.display import Image, display
from pathlib import Path

results_dir = Path("results")

# Training curves
print("=" * 60)
print("TRAINING CURVES")
print("=" * 60)
display(Image(filename=str(results_dir / "training_curves.png"), width=900))

In [ ]:
# Final samples
print("=" * 60)
print("FINAL GENERATED SAMPLES")
print("=" * 60)
display(Image(filename=str(results_dir / "final_samples.png"), width=900))

In [ ]:
# Class variation
print("=" * 60)
print("CLASS VARIATION (rows=fixed z, cols=class labels)")
print("=" * 60)
display(Image(filename=str(results_dir / "class_variation.png"), width=900))

In [ ]:
# Latent interpolation
print("=" * 60)
print("LATENT SPACE INTERPOLATION")
print("=" * 60)
display(Image(filename=str(results_dir / "latent_interpolation.png"), width=900))

In [ ]:
# Show samples at different epochs to see progression
import glob

print("=" * 60)
print("TRAINING PROGRESSION")
print("=" * 60)

sample_files = sorted(glob.glob(str(results_dir / "samples_epoch_*.png")))
for f in sample_files:
    epoch_str = Path(f).stem.replace("samples_epoch_", "")
    print(f"\nEpoch {int(epoch_str)}:")
    display(Image(filename=f, width=700))

## Step 5: Run full evaluation

This generates per-class sample grids, discriminator accuracy stats, and more.

In [ ]:
!python evaluation/evaluate.py \
    --checkpoint checkpoints/improved_final.pt \
    --config config/improved_config.yaml \
    --arch improved \
    --output_dir results/eval

In [ ]:
# View evaluation results
eval_dir = Path("results/eval")

print("CLASS VARIATION:")
display(Image(filename=str(eval_dir / "class_variation.png"), width=900))

print("\nLATENT INTERPOLATION:")
display(Image(filename=str(eval_dir / "latent_interpolation.png"), width=900))

## Step 6: Download results

Download the results and checkpoints to your local machine.

In [ ]:
# Zip results for download
!zip -r training_results.zip results/ checkpoints/improved_final.pt

from google.colab import files
files.download("training_results.zip")

## Optional: Mount Google Drive to save results persistently

If you want to save results across Colab sessions, mount your Google Drive.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Copy results to Drive
!cp -r results/ /content/drive/MyDrive/cgan_results/
!cp checkpoints/improved_final.pt /content/drive/MyDrive/cgan_results/
print("Results saved to Google Drive!")